# Segmenting and Clustering Neighborhoods in Toronto

## 1. Build the code to scrape the following Wikipedia page

In [2]:
!pip install bs4
!pip install geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=d81089f1a527989770691d19f09bab6b25e1f3f0c9f22c73b161e31f368de802
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
     |████████████████████████████████| 98 kB 7.1 MB/s  eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openm

mock-4.0.3           | 51 KB     | ##################################### | 100% 
jmespath-0.10.0      | 21 KB     | ##################################### | 100% 
pyasn1-0.4.8         | 53 KB     | ##################################### | 100% 
aiohttp-3.7.4        | 632 KB    | ##################################### | 100% 
libgfortran-ng-7.5.0 | 22 KB     | ##################################### | 100% 
async-timeout-3.0.1  | 11 KB     | ##################################### | 100% 
backcall-0.2.0       | 13 KB     | ##################################### | 100% 
astropy-4.2          | 7.5 MB    | ##################################### | 100% 
freetds-1.1.15       | 2.4 MB    | ##################################### | 100% 
libcblas-3.9.0       | 11 KB     | ##################################### | 100% 
gst-plugins-base-1.1 | 2.5 MB    | ##################################### | 100% 
regex-2020.11.13     | 365 KB    | ##################################### | 100% 
krb5-1.17.2          | 1.4 M

markdown-3.3.4       | 67 KB     | ##################################### | 100% 
matplotlib-3.3.4     | 6 KB      | ##################################### | 100% 
olefile-0.46         | 32 KB     | ##################################### | 100% 
snowballstemmer-2.1. | 57 KB     | ##################################### | 100% 
toml-0.10.2          | 18 KB     | ##################################### | 100% 
nbconvert-6.0.7      | 535 KB    | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
zeromq-4.3.4         | 352 KB    | ##################################### | 100% 
xz-5.2.5             | 343 KB    | ##################################### | 100% 
absl-py-0.12.0       | 96 KB     | ##################################### | 100% 
sip-4.19.25          | 295 KB    | ##################################### | 100% 
pyopenssl-20.0.1     | 48 KB     | ##################################### | 100% 
keras-applications-1 | 30 KB

sleef-3.5.1          | 1.5 MB    | ##################################### | 100% 
lcms2-2.12           | 443 KB    | ##################################### | 100% 
zfp-0.5.5            | 190 KB    | ##################################### | 100% 
mkl-service-2.3.0    | 54 KB     | ##################################### | 100% 
oauthlib-3.0.1       | 82 KB     | ##################################### | 100% 
seaborn-base-0.11.1  | 217 KB    | ##################################### | 100% 
soupsieve-2.0.1      | 30 KB     | ##################################### | 100% 
libwebp-base-1.2.0   | 808 KB    | ##################################### | 100% 
libzopfli-1.0.3      | 164 KB    | ##################################### | 100% 
py-1.10.0            | 73 KB     | ##################################### | 100% 
pygments-2.8.1       | 736 KB    | ##################################### | 100% 
lzo-2.10             | 314 KB    | ##################################### | 100% 
certifi-2020.12.5    | 143 K

done


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [12]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [13]:
soup = BeautifulSoup(html_data, 'html.parser')

## 2. Create the dataframe

In [14]:
data_pc = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood","Latitude","Longitude"])

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    
    if len(col)>0:
        pc =col[0].text.replace("\n", "")
        b =col[1].text.replace("\n", "")
        n =col[2].text.replace("\n", "")
    
        data_pc = data_pc.append({"PostalCode":pc, "Borough":b, "Neighborhood":n}, ignore_index=True)

data_pc = data_pc[data_pc.Borough != "Not assigned"]
data_pc = data_pc.groupby("PostalCode",as_index=False,sort=True).sum()
data_pc = data_pc.reset_index(drop=True)
# Chec if there is some not assigned neighborhoods
data_pc.loc[data_pc['Neighborhood'] == "Not assigned"]
# Show data shape
data_pc.shape

(103, 5)

## 3. Get longitude and latitude by useing Geocoder

In [15]:
import geocoder # import geocoder
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

In [16]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_fa9d07a7cae7449c9af7e35a6b5359de = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_fa9d07a7cae7449c9af7e35a6b5359de = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_fa9d07a7cae7449c9af7e35a6b5359de = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Mxq-DqyDnX4DL2AA_jSc2EqhiXtftDwnOOiWhqgPE1iv',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_fa9d07a7cae7449c9af7e35a6b5359de)

body = client_fa9d07a7cae7449c9af7e35a6b5359de.get_object(Bucket='datascience-donotdelete-pr-qg7cd0fgwpks0u',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
coordinates = pd.read_csv(body)

In [17]:
df = pd.DataFrame()
for index, row in data_pc.iterrows():
    postal = [row['PostalCode']]
    c_row  = coordinates[coordinates['Postal Code'].isin(postal)]
    latitude = c_row['Latitude'].values[0].astype(float)
    longitude = c_row['Longitude'].values[0].astype(float)
    row["Latitude"] = latitude
    row["Longitude"] = longitude
    
    df = df.append(row, ignore_index=True)

df.head(100)

,Borough,Latitude,Longitude,Neighborhood,PostalCode
0,Scarborough,43.806686,-79.194353,"Malvern, Rouge",M1B
1,Scarborough,43.784535,-79.160497,"Rouge Hill, Port Union, Highland Creek",M1C
2,Scarborough,43.763573,-79.188711,"Guildwood, Morningside, West Hill",M1E
3,Scarborough,43.770992,-79.216917,Woburn,M1G
4,Scarborough,43.773136,-79.239476,Cedarbrae,M1H
...,...,...,...,...,...
95,Etobicoke,43.643515,-79.577201,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",M9C
96,North York,43.756303,-79.565963,Humber Summit,M9L
97,North York,43.724766,-79.532242,"Humberlea, Emery",M9M
98,York,43.706876,-79.518188,Weston,M9N


## 4. Explore and cluster the neighborhoods in Toronto

### A. Generate Toronto map

In [18]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [19]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### B. Get Boroughs that only contains Toronto phrase

Explanation: I have decide to work only with boroughs that contain the word "Toronto"

In [43]:
df_toronto = pd.DataFrame()
for index, row in df.iterrows():

    if 'Toronto' in row["Borough"]:
        df_toronto = df_toronto.append(row, ignore_index=True)
df_toronto.shape
df_toronto.head(100)

,Borough,Latitude,Longitude,Neighborhood,PostalCode
0,East Toronto,43.676357,-79.293031,The Beaches,M4E
1,East Toronto,43.679557,-79.352188,"The Danforth West, Riverdale",M4K
2,East Toronto,43.668999,-79.315572,"India Bazaar, The Beaches West",M4L
3,East Toronto,43.659526,-79.340923,Studio District,M4M
4,Central Toronto,43.728020,-79.388790,Lawrence Park,M4N
5,Central Toronto,43.712751,-79.390197,Davisville North,M4P
6,Central Toronto,43.715383,-79.405678,"North Toronto West, Lawrence Park",M4R
7,Central Toronto,43.704324,-79.388790,Davisville,M4S
8,Central Toronto,43.689574,-79.383160,"Moore Park, Summerhill East",M4T
9,Central Toronto,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest...",M4V


In [29]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    -

### C. Generate map with markers

In [54]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for index, row in df_toronto.iterrows():
    marker_color= 'black'
    lng = row["Longitude"]
    lat = row["Latitude"]
    borough = row["Borough"]
    neighborhood = row["Neighborhood"]
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    if borough == "Central Toronto":
        marker_color= 'red'
    if borough == "West Toronto":
        marker_color= 'yellow'
    if borough == "East Toronto":
        marker_color= 'green'    
    if borough == "Downtown Toronto":
        marker_color= 'blue'        
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color= marker_color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### D. Observations

- there are 40 postal codes contains one or more neighborhoods
- there are 5 kinds of Boroughs in data: Central Toronto, East Toronto, West Toronto, Downtown Toronto and Toronto/York
- like we have 5 Boroughs we also have 5 dat clusters but one of them contains only one marker
- the smalest cluster is Toronto/York cluster
- the biggest cluster is Downtown Toronto cluster